In [7]:
using Parameters       # @unpack
using QuantEcon        # rouwenhorst
using LinearAlgebra    # dot
using Roots            # findzero
using SparseArrays     # SparseMatrixCSC
using IterativeSolvers # powm!
using Plots            # Plots
using NBInclude        # including notebooks instead of jl files
using MAT              ####  write the .m file 
using DataFrames
using Printf


In [8]:
@nbinclude("Structures_SP.ipynb");
@nbinclude("Utils_SP.ipynb");
@nbinclude("SolveAiyagari_SP.ipynb");
@nbinclude("Projection_SP.ipynb");
@nbinclude("Projection_SP_ref.ipynb");
@nbinclude("Ramsey_SP.ipynb");

## Main Specification 

In [9]:

#########################
#### MAIN SPECIFICATION 
#########################


params = Params(
    KsY=2.5*4,
    α=0.36,  # capital share
    δ=0.025, # capital depreciation
    γ=1.,   
    ϕ=0.5,   # Frisch elasticity 0.5
    χ = 1.0, 
    ϵ = 6.0,
    κ= 100.0, ####  price adjustment not taxe
    ν= 0.0,   #### inequal distribution of profit 
    tl=0.28,  #### tax rate labor 
    tk=0.36,  #### tax rate capital
    Tt=0.114, #### transfert
    na=100,
    a_min=1e-9, 
    a_max=1000.0,
    curv_a=4.0,
    ny=5,
    ρy= 0.99, # 
    σy= 0.123 # 
    );

solution = steady(params);

N=5
proj = Project_plan(N,solution,params); 
plan = multi_find(proj,solution, params);
Write_Dynare(proj,solution,params,plan,"baseline")
describe_solution(solution,params)


iteration: 50 0.09568587046056519
iteration: 100 0.030022482274856398
iteration: 150 0.007256960017945231
iteration: 200 0.002846844148156604
iteration: 250 0.0016444734011664863
iteration: 300 0.0007640979840733611
iteration: 350 0.00031200000002817835
iteration: 400 0.00011848289549780579
iteration: 450 4.3149783452055614e-5
iteration: 500 1.532034875746691e-5
iteration: 550 5.355401964374852e-6
iteration: 600 1.855284204600897e-6
iteration: 650 6.39903051504716e-7
iteration: 700 2.2040068142486963e-7
iteration: 750 7.593797360139109e-8
iteration: 800 2.6193087600748188e-8
iteration: 850 9.04609362296012e-9
Solved in 850 iterations
sum(y0_h .^ ν .* S_h) - sum(ys .^ ν .* Sy) = 5.241918010767677e-12


Dict{String, Float64} with 11 entries:
  "total tax/Y"                 => 0.228864
  "Share of constrained agents" => 0.194328
  "L"                           => 0.391889
  "MPC"                         => 0.172001
  "I/Y"                         => 0.25
  "C/Y"                         => 0.628753
  "Gini"                        => 0.730188
  "G/Y"                         => 0.121247
  "B/Y"                         => 0.635375
  "Transfers/Y"                 => 0.0796604
  "K/Y"                         => 2.5

In [10]:
################################################
#### TABLE 2: Wealth distribution in the model
################################################
@unpack aGrid,ny= params

ww = [repeat(params.aGrid,params.ny) vec(solution.stationaryDist)] ;
ww2 = sortslices(ww, dims=1) ; 
vW = round.(wealthD(ww2,5), digits=2) ;

# Table 2 
df_quartile= DataFrame(Quartile=vec(round.(wealthD(ww2,5)[:,2]*100, digits=1)) ) ;
df_Gini= DataFrame(Gini = round.(Gini(solution.ga, solution.stationaryDist),digits=2)) ;
df_Top5= DataFrame(Top5 =round.(wealthD(ww2,20)[:,2]*100, digits=1)[end]) ;

display(df_quartile)
display(df_Top5)
display(df_Gini)

Row,Quartile
,Float64
1,0.0
2,0.2
3,5.2
4,18.2
5,76.4


Row,Top5
,Float64
1,33.2


Row,Gini
,Float64
1,0.73


### APPENDIX D7: The refined truncation


In [11]:
##################################################
#### APPENDIX D7 The refined truncation
##################################################
@nbinclude("Projection_SP_ref.ipynb");
N=5;
Vidio = repeat([20],params.ny)

proj_refined = refinedProject_plan(N,Vidio,solution,params,new_ξs=false)
plan_refined = multi_find(proj_refined,solution, params)
Write_Dynare(proj_refined,solution,params,plan_refined,"refined")

sum(y0_h .^ ν .* S_h) - sum(ys .^ ν .* Sy) = 5.241918010767677e-12


### Appendix E1: generate steady-state for robustness on the slope 

In [ ]:

# #########################
# #### Robustness 
# #########################

slope_p = [0.0062, 0.019, 0.05 ,0.085, 0.24] ;
kappa_rob = (params.ϵ-1) ./slope_p ;  

for kappa_params in kappa_rob 

    params = Params(
        KsY=2.5*4,
        α=0.36,  # capital share
        δ=0.025, # capital depreciation
        γ=1.,   
        ϕ=0.5,  # Frisch elasticity 0.5
        χ = 1.0, # defined later 
        ϵ = 6.0,
        κ= kappa_params, ####  price adjustment not taxe
        ν= 0.0, #### inequal distribution of profit 
        tl=0.28, #### tax rate labor 
        tk=0.36, #### tax rate capital
        Tt=0.114, #### transfert
        na=100,
        a_min=1e-9, 
        a_max=1000.0,
        curv_a=4.0,
        ny=5,
        ρy= 0.99, # 
        σy= 0.123 # 
        );

    solution = steady(params);

    N=5
    proj = Project_plan(N,solution,params); 
    plan = multi_find(proj,solution, params);

    formatted_with_decimals = @sprintf("%.2f", kappa_params)
    kappa_int = Int(floor(parse(Float64, formatted_with_decimals)))
    formatted_name = @sprintf("%d", kappa_int)
    Write_Dynare(proj,solution,params,plan,"rob_$(formatted_name)")
end

### APPENDIX E.3.2 Unequal profit distribution


In [12]:
##################################################
#### APPENDIX E.3.2 Unequal profit distribution
##################################################

params_unequal = Params(
    KsY=2.5*4,
    α  = 0.36,  
    δ  =0.025, 
    γ  =1.,  ####  
    ϕ  =0.5,  
    χ  =1.0,
    ϵ  =6.0,
    κ  =20.0,####  the Rotemberg cost parameter is set to =20
    ν  =2.3, #### the parameter how unequal the profit distribution is ; set to =2.3 
    tl =0.28,
    tk =0.36, 
    Tt =0.114, 
    na =100,
    a_min=1e-9, 
    a_max=1000.0,
    curv_a=4.0,
    ny=5,
    ρy= 0.99,
    σy= 0.123
    );

solution_unequal = steady(params_unequal);


N=5
proj_unequal = Project_plan(N,solution_unequal,params_unequal);
plan_unequal = multi_find(proj_unequal,solution_unequal, params_unequal);
Write_Dynare(proj_unequal,solution_unequal,params_unequal,plan_unequal,"unequal")
describe_solution(solution_unequal,params_unequal)



iteration: 50 0.09568587046056519
iteration: 100 0.030022482274856398
iteration: 150 0.007256960017945231
iteration: 200 0.002846844148156604
iteration: 250 0.0016444734011664863
iteration: 300 0.0007640979840733611
iteration: 350 0.00031200000002817835
iteration: 400 0.00011848289549780579
iteration: 450 4.3149783452055614e-5
iteration: 500 1.532034875746691e-5
iteration: 550 5.355401964374852e-6
iteration: 600 1.855284204600897e-6
iteration: 650 6.39903051504716e-7
iteration: 700 2.2040068142486963e-7
iteration: 750 7.593797360139109e-8
iteration: 800 2.6193087600748188e-8
iteration: 850 9.04609362296012e-9
Solved in 850 iterations
sum(y0_h .^ ν .* S_h) - sum(ys .^ ν .* Sy) = 1.2830625450988009e-11


Dict{String, Float64} with 11 entries:
  "total tax/Y"                 => 0.228864
  "Share of constrained agents" => 0.194328
  "L"                           => 0.391889
  "MPC"                         => 0.172001
  "I/Y"                         => 0.25
  "C/Y"                         => 0.628753
  "Gini"                        => 0.730188
  "G/Y"                         => 0.121247
  "B/Y"                         => 0.635375
  "Transfers/Y"                 => 0.0796604
  "K/Y"                         => 2.5